In [1]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import reconnect_degree_2
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
import os
from matplotlib import colors
from random import choice
from experiment_class import Experiment,clean_exp_with_hyphaes
from hyphae_id import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends, solve_degree4, clean_obvious_fake_tips
from extract_graph import prune_graph



In [6]:
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from pycpd import RigidRegistration, DeformableRegistration
import open3d as o3d
plate = 9
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[0:40]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
dilateds=[]
skels = []
skel_docs = []
graph_pos=[]
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    try:
        skel = read_mat(path_snap+'/Analysis/skeleton.mat')['skeleton']
    except:
        print(date)

0714_1415
0715_0334


In [8]:
len(list_dir_interest)

48

In [2]:
plate = 130
exp_clean = pickle.load(open( f'Data/'+f"experiment_{plate}.pick", "rb" ) )

In [3]:
len(exp_clean.hyphaes)

2621

In [4]:
solve_two_ends = resolve_ambiguity_two_ends(exp_clean.hyphaes)
solved,solved_nodes = solve_degree4(exp_clean)

240
[(595737.0, 2399)]
[(609538.0, 2469)]
[(634928.0, 2577)]
[(646101.0, 2607)]
[(676248.0, 2704)]
[(705415.0, 2837)]
[(736676.0, 2973)]
[(775211.0, 3115)]
[(803529.0, 3227)]
[(837312.0, 3366)]
[(864091.0, 3493)]
[(881091.0, 3547)]
[(904837.0, 3665)]
[(954177.0, 3806)]
[(975529.0, 3904)]
[(995997.0, 4002)]
[(1025924.0, 4179)]
[(1054089.0, 4311)]
[(1090754.0, 4479)]
[(1095056.0, 4491)]
[(1105711.0, 4538)]
[(1127468.0, 4626)]
[(1146774.0, 4843)]
46


In [11]:
len(exp_clean.hyphaes)

2194

In [8]:
plt.close('all')
node = choice(exp_clean.nodes)
t0 = choice(node.ts())
exp_clean.plot([22],[[]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
len(solved_nodes)

46

In [9]:
clean_obvious_fake_tips(exp_clean)

[(592239.0, 2355)]
[(608429.0, 2450)]
[(633824.0, 2555)]
[(644730.0, 2582)]
[(675716.0, 2690)]
[(704642.0, 2827)]
[(735586.0, 2949)]
[(774401.0, 3104)]
[(802903.0, 3210)]
[(835239.0, 3349)]
[(862807.0, 3478)]
[(880482.0, 3533)]
[(903735.0, 3635)]
[(953918.0, 3797)]
[(974862.0, 3888)]
[(995182.0, 3990)]
[(1025195.0, 4167)]
[(1053697.0, 4300)]
[(1085771.0, 4434)]
[(1094208.0, 4478)]
[(1105389.0, 4521)]
[(1125479.0, 4598)]
[(1146774.0, 4843)]


In [10]:
to_remove = []
for hyph in exp_clean.hyphaes:
    hyph.update_ts()
    if len(hyph.ts)==0:
        to_remove.append(hyph)
for hyph in to_remove:
    exp_clean.hyphaes.remove(hyph)

In [8]:
have_mother = [hyph for hyph in exp_clean.hyphaes if len(hyph.mother)>=1]

In [9]:
have_mother_2 = [hyph for hyph in exp_clean.hyphaes if len(hyph.mother)>=2]

In [41]:
len(have_mother_2)

274

In [37]:
len(have_mother)

1760

In [10]:
new_hyphaes_t={t : [] for t in range(len(exp_clean.nx_graph))}
for hyphae in exp_clean.hyphaes:
    new_hyphaes_t[hyphae.end.ts()[0]].append(hyphae)

In [11]:
num_new = {t : len(new_hyphaes_t[t]) for t in new_hyphaes_t.keys()}

In [12]:
distance_from_mother = {}
for hyph in have_mother:
    t0=hyph.ts[0]
    if t0 in hyph.mother[0].end.ts():
        distance_from_mother[hyph] = np.linalg.norm(hyph.root.pos(t0)-hyph.mother[0].end.pos(t0))

In [13]:
plt.close('all')
hyph = choice(list(distance_from_mother.keys()))
print(distance_from_mother[hyph])
exp_clean.plot([hyph.ts[0]],[[hyph.end.label,hyph.root.label,hyph.mother[0].end.label]])

110.01818031580054


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
sub_apical_branch = [hyph for hyph in distance_from_mother.keys() if distance_from_mother[hyph]>=500 and hyph.ts[0]>=1]

In [34]:
len(sub_apical_branch)

265

In [18]:
plt.close('all')
hyph = choice(sub_apical_branch)
exp_clean.plot([hyph.ts[0]-1,hyph.ts[0]],[[hyph.end.label,hyph.root.label]]*2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(np.log10(np.array(list(distance_from_mother.values()))+1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 56.,   9.,  37., 143., 287., 525., 326., 205., 116.,  19.]),
 array([0.        , 0.42364323, 0.84728645, 1.27092968, 1.69457291,
        2.11821614, 2.54185936, 2.96550259, 3.38914582, 3.81278905,
        4.23643227]),
 <BarContainer object of 10 artists>)

In [52]:
max(distance_from_mother.values())

17234.83289736225

In [19]:
from hyphae_id import get_pixel_growth_and_new_children
apical_branching = []
for hyph in exp_clean.hyphaes:
    for index in range(len(hyph.ts[:-1])):
        t = hyph.ts[index]
        tp1 = hyph.ts[index+1]
        pixels,nodes = get_pixel_growth_and_new_children(hyph,t,tp1)
        for node in nodes[1:-1]:
            apical_branching.append((t,node))

            
        

In [23]:
plt.close('all')
t0,node = choice(apical_branching)
print(node.ts())
exp_clean.plot([t0,t0+1],[[node.label]]*2)

[8, 9, 10, 11, 12, 13, 14, 15, 16, 17]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
subapical_branching = [(hyph.root.ts()[0]-1,hyph.root) for hyph in exp_clean.hyphaes if (hyph.root.ts()[0]-1,hyph.root) not in apical_branching and hyph.root.ts()[0]>=1]

In [25]:
len(apical_branching)

512

In [31]:
plt.close('all')
t0,node = choice(subapical_branching)
print(node.ts())
exp_clean.plot([t0,t0+1],[[node.label]]*2)

[18, 19, 20, 21, 22]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
exp_clean.plot([t0,t0+1,t0+2],[[node.label]]*3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
node.neighbours(t0+1)

[Node(4707), Node(3453), Node(4024)]

In [39]:
len(exp_clean.hyphaes)

2194

In [101]:
plt.close('all')
t0,node = choice(apical_branching)
exp_clean.plot([t0,t0+1],[[node.label]]*2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
plt.close('all')
t = 6
exp_clean.plot([t-1,t,t+1],[[hypha.end.label for hypha in new_hyphaes_t[t]]]*3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
number_face=[2+len(g.edges)-len(g.nodes) for g in exp_clean.nx_graph]

In [28]:
number_anas_theory = [number_face[i+1]-number_face[i] for i in range(len(number_face)-2)]

In [29]:
for hyph in exp_clean.hyphaes:
    hyph.update_ts()

In [30]:
hyph_anas_tip_hyph = [hyphat for hyphat in exp_clean.hyphaes if len(hyphat.ts)>=1 and hyphat.end.degree(hyphat.ts[-1])>=3 and hyphat.end.degree(hyphat.ts[-2])>=3]
hyph_anas_tip_tip = []
potential = []
for hyph in exp_clean.hyphaes:
    if len(hyph.ts)>=2 and hyph.end.degree(hyph.ts[-1])==1 and hyph.end.ts()[-1]!=len(exp_clean.nx_graph)-1 and not np.all([hyph.get_length_pixel(t)<=20 for t in hyph.ts]):
        potential.append(hyph)
for hyph in potential:
    t0 = hyph.ts[-1]
    for hyph2 in potential:
        if hyph2.ts[-1] == t0 and hyph!=hyph2:
            vector = (hyph2.end.pos(t0)-hyph.end.pos(t0))/np.linalg.norm(hyph2.end.pos(t0)-hyph.end.pos(t0))
            vertical_vector=np.array([-1,0])
            dot_product = np.dot(vertical_vector,vector)
            if vertical_vector[1]*vector[0]-vertical_vector[0]*vector[1]>=0: #determinant
                angle = np.arccos(dot_product)/(2*np.pi)*360
            else:
                angle = -np.arccos(dot_product)/(2*np.pi)*360
            score = np.cos((angle-(180+hyph.end.edges(t0)[0].orientation_begin(t0,30)))/360*2*np.pi)+np.cos((360+angle-hyph2.end.edges(t0)[0].orientation_begin(t0,30))/360*2*np.pi)
            if np.linalg.norm(hyph2.end.pos(t0)-hyph.end.pos(t0))<=500 and score>=0.5:
                hyph_anas_tip_tip.append((hyph,hyph2,t0))

In [31]:
len(hyph_anas_tip_hyph)

293

In [32]:
len(hyph_anas_tip_tip)

68

In [33]:
len(exp_clean.hyphaes)

3595

In [35]:
exp_clean.plot([10],[[hyph.end.label,hyph.root.label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
def get_time_anas_tip_hyph(anas_tip_hyph):
    for t in anas_tip_hyph.ts:
        if anas_tip_hyph.end.degree(t)>=3:
            return(t-1)
def get_time_anas_tip_tip(anas_tip_tip):
    return(anas_tip_tip.ts[-1])

In [13]:
anas_tip_hyph_observed = np.array([len([hyph for hyph in hyph_anas_tip_hyph if get_time_anas_tip_hyph(hyph)==t]) for t in range(len(exp_clean.nx_graph)-2)])

In [14]:
anas_tip_tip_observed = np.array([len([hyphs for hyphs in hyph_anas_tip_tip if hyphs[2]==t])//2 for t in range(len(exp_clean.nx_graph)-2)])

In [62]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(anas_tip_hyph_observed)
ax.plot(number_anas_theory)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
plt.close('all')
hyph,hyph2,t0 = choice(hyph_anas_tip_tip)

In [45]:
hyph.ts

[12, 13, 14, 15]

In [46]:
exp_clean.plot([t0,t0+1],[[hyph.end.label,hyph.root.label,hyph2.end.label,hyph2.root.label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
hyph2.root.show_source_image(t0,t0+1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
hyph_tip_tip = choice(hyph_anas_tip_hyph)

In [47]:
plt.close('all')
t0 = get_time_anas_tip_hyph(hyph_tip_tip)
exp_clean.plot([t0,t0+1],[[hyph_tip_tip.end.label,hyph_tip_tip.end.label]]*2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
exp_test = Experiment(13)
exp_test.copy(exp_clean)
exp_test.get_node(hyph_tip_tip.end.label).show_source_image(t0,t0+1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …